In [1]:
import numpy as np
import pandas as pd
import pickle
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
dataset = 'stv2'

## load from pkl matrix

In [3]:
# open pickle file in read mode
with open('/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/thesis/co_exp/st_v2.pkl', 'rb') as f:
    # load data from pickle file
    st = pickle.load(f)
st_arr = st[0]
st_idx= st[1]

In [6]:
import scipy.sparse as sp

# create sparse matrix from st_arr_new
sparse_matrix = sp.csr_matrix(st_arr)

# get upper triangle as a sparse array
sparse_upper = sp.triu(sparse_matrix, k=1)
i, j, data = sp.find(sparse_upper)

In [12]:
st_idx = np.array(st_idx)
col1 = st_idx[i]

In [14]:
df2 = pd.DataFrame({'combined_score': data})
df2['protein1'] = col1

In [5]:
import scipy.sparse as sp

# create sparse matrix from st_arr_new
sparse_matrix = sp.csr_matrix(st_arr)

# get upper triangle as a sparse array
sparse_upper = sp.triu(sparse_matrix, k=1)
i, j, data = sp.find(sparse_upper)

col1 = st_idx[i]
col2 = st_idx[j]
df2 = pd.DataFrame({'combined_score': data})
df2['protein1'] = col1.values
df2['protein2'] = col2.values
df2['combined_score'] = df2['combined_score'].round(3)

## PPI inputs

In [3]:
local_stringdb = '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/data/string/lfs-stringdb/'
# load local STRING database and names
df = pd.read_csv(local_stringdb+'9606.protein.info.v12.0.txt', sep='\t', header=0, usecols=['#string_protein_id', 'preferred_name'])
df['preferred_name'] = df['preferred_name'].str.upper()
stringId2name = df.set_index('#string_protein_id')['preferred_name'].to_dict()
name2stringId = df.set_index('preferred_name')['#string_protein_id'].to_dict()
dataset = 'ppi'
graph_df = pd.read_csv(local_stringdb+'9606.protein.physical.links.detailed.v12.0.txt', sep=' ', header=0).convert_dtypes().replace(0, float('nan'))
graph_df = graph_df[['protein1', 'protein2','combined_score']]
graph_df['protein1']=graph_df['protein1'].map(stringId2name)
graph_df['protein2']=graph_df['protein2'].map(stringId2name)

## load from existing csv file

In [2]:

graph_df = pd.read_csv('/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/thesis/data/st_all.csv')

# to graph

In [19]:
graph_df  = graph_df.rename(columns={'coexp_value': 'combined_score', 'gene1': 'protein1', 'gene2': 'protein2'})

In [3]:
graph_df = graph_df.sort_values(by='coexp_value',ascending=True)

In [5]:
graph_df.tail(3)

,coexp_value,gene1,gene2
90450158,0.823,UBC,MYL9
42835709,0.824,ACTA2,UBC
106791046,0.830,UBC,FLNA


In [7]:
proteins = sorted(list(set(graph_df['gene1'].tolist())|set(graph_df['gene2'].tolist())))
gene2node = {value: index for index, value in enumerate(proteins)}

file_path = '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/thesis/HBDM/data/datasets/'+dataset+'/ppi_index.pkl'
# Serialize and save the Tensor to the file
with open(file_path, 'wb') as file:
    pickle.dump(gene2node, file)
# Close the file
file.close()

In [9]:
graph_df['node1']=graph_df['gene1'].map(gene2node)
graph_df['node2']=graph_df['gene2'].map(gene2node)

In [11]:
G = nx.from_pandas_edgelist(graph_df, source='node1', target='node2', edge_attr='coexp_value', create_using=nx.Graph)

In [7]:
edges = np.array([(u, v, data['combined_score']) for u, v, data in G.edges(data=True)])

# Determine i, j, and weights
i = np.where(edges[:, 0] > edges[:, 1], edges[:, 1], edges[:, 0])
j = np.where(edges[:, 0] > edges[:, 1], edges[:, 0], edges[:, 1])
#weights = edges[:, 2]

root = '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/thesis/HBDM/data/datasets/'+dataset+'/'
np.savetxt(root+'sparse_i.txt', np.array(i), delimiter='\n')
np.savetxt(root+'sparse_j.txt', np.array(j), delimiter='\n')
np.savetxt(root+'sparse_w.txt', np.array(weights), delimiter='\n')

In [27]:
graph_df[(graph_df['node1']==12563)&(graph_df['node2'] == 9357)]

,combined_score,protein1,protein2,node1,node2
0,0.001,SAMD11,NOC2L,12563,9357


In [34]:

int(str(0.5*10)[0])

5

In [25]:
level_edges

{1: [[12563, 9357],
  [12563, 1268],
  [12563, 15753],
  [12563, 127],
  [12563, 4000],
  [12563, 16130],
  [12563, 15018],
  [12563, 8432],
  [12563, 2278],
  [12563, 8968],
  [12563, 10270],
  [12563, 15172],
  [12563, 11295],
  [12563, 15306],
  [12563, 16283],
  [12563, 7843],
  [12563, 2384],
  [12563, 9427],
  [12563, 6969],
  [12563, 6434],
  [12563, 10919],
  [12563, 9374],
  [12563, 16431],
  [12563, 3799],
  [12563, 1755],
  [12563, 10576],
  [12563, 13270],
  [12563, 10763],
  [12563, 3215],
  [12563, 11994],
  [12563, 15775],
  [12563, 3619],
  [12563, 10589],
  [12563, 1833],
  [12563, 4529],
  [12563, 8774],
  [12563, 15777],
  [12563, 4874],
  [12563, 423],
  [12563, 2651],
  [12563, 10947],
  [12563, 8459],
  [12563, 15177],
  [12563, 16076],
  [12563, 6951],
  [12563, 3804],
  [12563, 3499],
  [12563, 16406],
  [12563, 4362],
  [12563, 9137],
  [12563, 9023],
  [12563, 1144],
  [12563, 12719],
  [12563, 12010],
  [12563, 6543],
  [12563, 514],
  [12563, 6457],
  [12563

In [24]:
level_edges = dict()
for u, v, data in G.edges(data=True):
    level = int(str(data['combined_score']*10)[0])
    if level in level_edges:
        level_edges[level].append([u, v])
    else:
        level_edges[level]=[[u, v]]

KeyboardInterrupt: 

In [8]:
level_edges = dict()
for u, v, data in G.edges(data=True):
    level = int(str(data['combined_score']*1000)[0])
    if level in level_edges:
        level_edges[level].append([u, v])
    else:
        level_edges[level]=[[u, v]]

for level in level_edges:
    edges = np.array(level_edges[level])
    sparse_i = np.where(edges[:, 0] > edges[:, 1], edges[:, 1], edges[:, 0])
    sparse_j = np.where(edges[:, 0] > edges[:, 1], edges[:, 0], edges[:, 1])
    np.savetxt(root+'level_'+str(level)+'_sparse_i.txt', np.array(sparse_i), delimiter='\n')
    np.savetxt(root+'level_'+str(level)+'_sparse_j.txt', np.array(sparse_j), delimiter='\n')

KeyboardInterrupt: 

In [ ]:

def plot_degree_distribution(name, G, use_weight=False):
    """
    Plot the degree distribution of a graph.

    Parameters:
    - G: NetworkX graph
    - use_weight: Boolean, whether to consider edge weights in the degree calculation
    """

    # Compute the degree distribution
    if use_weight:
        print('weight')
        degree_sequence = sorted([d for n, d in G.degree(weight="weight")], reverse=True)
    else:
        degree_sequence = sorted([d for n, d in G.degree()], reverse=True)

    degree_counts = nx.degree_histogram(G)

    # Convert counts to fractions
    num_nodes = len(G.nodes)
    degree_fraction = [count / num_nodes for count in degree_counts]

    # Plot the degree distribution on a log-log scale
    plt.loglog(range(len(degree_fraction)), degree_fraction, 'o', markersize=5)
    plt.title(name+" Degree Distribution")
    plt.xlabel("Degree (log scale)")
    plt.ylabel("Fraction of Nodes (log scale)")
    plt.show()

def average_degree(G, use_weight=False):
    """
    Calculate the average degree of a graph.

    Parameters:
    - G: NetworkX graph
    - use_weight: Boolean, whether to consider edge weights in the degree calculation

    Returns:
    - avg_degree: Average degree of the graph
    """

    # Compute the degree sequence
    if use_weight:
        degree_sequence = [d for n, d in G.degree(weight="weight")]
    else:
        degree_sequence = [d for n, d in G.degree()]

    # Calculate the average degree
    avg_degree = sum(degree_sequence) / len(degree_sequence)

    return avg_degree

#plot_degree_distribution('ppi-weight',G, use_weight=True)